In [1]:
import numpy as np
import pandas as pd
from wordcloud import WordCloud
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import random
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.types as t
import pyspark.sql.functions as f
import seaborn as sns
import pickle
from pyspark.sql.functions import col, sum, count
import dataframe_image as dfi

sns.set(style="darkgrid")

file_path = "/home/olek/studia/semestr_3_AM/Big_Data/Big_Data/projekt/data/charts.csv"
# file_path = "/home/olek/studia/semestr_3_AM/Big_Data/Big_Data/projekt/data/sample.csv"

# results_path = "/home/olek/studia/semestr_3_AM/Big_Data/Big_Data/projekt/sample_results/"
if file_path.endswith("sample.csv"):
    results_path = "/home/olek/studia/semestr_3_AM/Big_Data/Big_Data/projekt/sample_results/"
    images_path = "/home/olek/studia/semestr_3_AM/Big_Data/Big_Data/projekt/sample_images/"
else:
    results_path = "/home/olek/studia/semestr_3_AM/Big_Data/Big_Data/projekt/results/"
    images_path = "/home/olek/studia/semestr_3_AM/Big_Data/Big_Data/projekt/images/"

seed = 777
np.random.seed(seed)
random.seed(seed)

In [12]:
# pd.read_csv("/home/olek/studia/semestr_3_AM/Big_Data/Big_Data/projekt/data/charts.csv", nrows=10**6)\
#     .to_csv("/home/olek/studia/semestr_3_AM/Big_Data/Big_Data/projekt/data/sample.csv", index=False)

In [2]:
spark = SparkSession.builder \
    .appName("Results") \
    .getOrCreate()
df = spark.read.csv(file_path, header=True, inferSchema=True)
df = df.withColumn("rank", f.col("rank").cast(t.IntegerType())).withColumn("date", f.col("date").cast(t.DateType())).withColumn("streams", f.col("streams").cast(t.IntegerType()))
df.registerTempTable("charts")

24/06/25 01:24:07 WARN Utils: Your hostname, olek resolves to a loopback address: 127.0.1.1; using 192.168.100.7 instead (on interface wlp6s0)
24/06/25 01:24:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/25 01:24:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/olek/.local/lib/python3.10/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [3]:
df.printSchema()

root
 |-- title: string (nullable = true)
 |-- rank: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- artist: string (nullable = true)
 |-- url: string (nullable = true)
 |-- region: string (nullable = true)
 |-- chart: string (nullable = true)
 |-- trend: string (nullable = true)
 |-- streams: integer (nullable = true)



# Show head data:

In [4]:
head = spark.sql("SELECT * FROM charts LIMIT 10")
with open(results_path + "head.pkl", "wb") as f:
    pickle.dump(head.toPandas(), f)
    
df_styled = spark.sql("SELECT * from charts limit 10").toPandas().style.background_gradient() 
dfi.export(df_styled, images_path + "mytable.png")
df_styled = head.toPandas().style.background_gradient() 
dfi.export(df_styled, images_path + "head.png")
head.toPandas()

,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956
5,Traicionera,6,2017-01-01,Sebastian Yatra,https://open.spotify.com/track/5J1c3M4EldCfNxX...,Argentina,top200,MOVE_DOWN,151140
6,Cuando Se Pone a Bailar,7,2017-01-01,Rombai,https://open.spotify.com/track/1MpKZi1zTXpERKw...,Argentina,top200,MOVE_DOWN,148369
7,Otra vez (feat. J Balvin),8,2017-01-01,Zion & Lennox,https://open.spotify.com/track/3QwBODjSEzelZyV...,Argentina,top200,MOVE_DOWN,143004
8,La Bicicleta,9,2017-01-01,"Carlos Vives, Shakira",https://open.spotify.com/track/0sXvAOmXgjR2QUq...,Argentina,top200,MOVE_UP,126389
9,Dile Que Tu Me Quieres,10,2017-01-01,Ozuna,https://open.spotify.com/track/20ZAJdsKB5IGbGj...,Argentina,top200,MOVE_DOWN,112012


# EDA

## Missing data

In [5]:
# 3min 32s
columns = df.columns
missing_data = df.select([(sum(col(c).isNull().cast("int")).alias(c)) for c in columns])
missing_data.show()

with open(results_path + "missing_data.pkl", "wb") as f:
    pickle.dump(missing_data.toPandas(), f)

df_styled = missing_data.toPandas().style.background_gradient()
dfi.export(df_styled, images_path + "missing_data.png")

+-----+----+----+------+---+------+-----+-----+-------+
|title|rank|date|artist|url|region|chart|trend|streams|
+-----+----+----+------+---+------+-----+-----+-------+
|    0|2142|2142|     0|  0|     0|    0|    0|5855331|
+-----+----+----+------+---+------+-----+-----+-------+



## unique_values

In [7]:
%time
# time 8min 26s
missing_streams_df = df.filter(col('streams').isNull())
unique_values = {col_name: missing_streams_df.select(col_name).distinct().count() for col_name in missing_streams_df.columns}
unique_values_df = pd.DataFrame(unique_values, index=[0])

with open(results_path + "unique_values_df.pkl", "wb") as f:
    pickle.dump(unique_values_df, f)

df_styled = unique_values_df.style.background_gradient() 
dfi.export(df_styled, images_path + "unique_values.png")
unique_values_df

CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 12.6 µs


,title,rank,date,artist,url,region,chart,trend,streams
0,119407,201,1827,81938,148226,208,102,68,1


## Count np of missing striming value per chart

In [8]:
%time
null_cart = spark.sql("""
    SELECT chart, count(chart) as count
    FROM charts
    WHERE streams IS NULL
    GROUP BY chart
    ORDER BY count DESC;""")
null_cart_df = null_cart.toPandas()
with open(results_path + "null_cart_df.pkl", "wb") as f:
    pickle.dump(null_cart_df, f)

# df_styled = null_cart_df.style.background_gradient() 
# dfi.export(df_styled, images_path + "unique_values.png")
null_cart_df

CPU times: user 14 µs, sys: 4 µs, total: 18 µs
Wall time: 498 µs


ValueError: Your Styled DataFrame has more than 100 rows and will produce a huge image file, possibly causing your computer to crash. Override this error by explicitly setting `max_rows` to -1 for all columns. Styled DataFrames are unable to select a subset of rows or columns and therefore do not work with the `max_rows` and `max_cols` parameters

,chart,count
0,viral50,5848001
1,Paraguay,704
2,Germany,555
3,Chile,516
4,https://open.spotify.com/track/0xsAtU9TemtJgyV...,438
...,...,...
97,"Cosculluela""",1
98,Russia,1
99,https://open.spotify.com/track/70wb5cVwwdHzvpt...,1
100,https://open.spotify.com/track/4p80VsLJul7HDkx...,1


# SElect TOP200 

In [3]:
%time
filtered_df = spark.sql("SELECT * FROM charts WHERE chart = 'top200'")
filtered_df.show()
df = filtered_df
df.createOrReplaceTempView("charts")

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 14.3 µs


+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+
|               title|rank|      date|              artist|                 url|   region| chart|        trend|streams|
+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+
|Chantaje (feat. M...|   1|2017-01-01|             Shakira|https://open.spot...|Argentina|top200|SAME_POSITION| 253019|
|Vente Pa' Ca (fea...|   2|2017-01-01|        Ricky Martin|https://open.spot...|Argentina|top200|      MOVE_UP| 223988|
|Reggaetón Lento (...|   3|2017-01-01|                CNCO|https://open.spot...|Argentina|top200|    MOVE_DOWN| 210943|
|              Safari|   4|2017-01-01|J Balvin, Pharrel...|https://open.spot...|Argentina|top200|SAME_POSITION| 173865|
|         Shaky Shaky|   5|2017-01-01|        Daddy Yankee|https://open.spot...|Argentina|top200|      MOVE_UP| 153956|
|         Traicionera|   6|2017-01-01|  

In [45]:
%time
spark.sql('''
SELECT COUNT(*) NoOfObservationsTop200
FROM charts;
''').toPandas().head(10)

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 13.8 µs


,NoOfObservationsTop200
0,20318183


In [10]:
%time
columns = df.columns
missing_data = df.select([(sum(col(c).isNull().cast("int")).alias(c)) for c in columns])
missing_data.show()

with open(results_path + "missing_data_clearing.pkl", "wb") as f:
    pickle.dump(missing_data.toPandas(), f)

df_styled = missing_data.toPandas().style.background_gradient()
dfi.export(df_styled, images_path + "missing_data.png")

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.82 µs


+-----+----+----+------+---+------+-----+-----+-------+
|title|rank|date|artist|url|region|chart|trend|streams|
+-----+----+----+------+---+------+-----+-----+-------+
|    0|   0|   0|     0|  0|     0|    0|    0|      0|
+-----+----+----+------+---+------+-----+-----+-------+



In [11]:
%time
data_intervale = spark.sql("SELECT  MAX(date), MIN(date), MAX(date) - MIN(date) as intervale FROM charts")
data_intervale.show()
data_intervale_df = data_intervale.toPandas() 
full_data = data_intervale_df["intervale"][0].days

with open(results_path + "data_intervale.pkl", "wb") as f:
    pickle.dump(data_intervale_df, f)

df_styled = data_intervale.toPandas()
df_styled["intervale"] = full_data
df_styled.style.background_gradient()
dfi.export(df_styled, images_path + "data_intervale.png")
df_styled

CPU times: user 11 µs, sys: 4 µs, total: 15 µs
Wall time: 25.7 µs


+----------+----------+-------------------+
| max(date)| min(date)|          intervale|
+----------+----------+-------------------+
|2021-12-31|2017-01-01|INTERVAL '1825' DAY|
+----------+----------+-------------------+



,max(date),min(date),intervale
0,2021-12-31,2017-01-01,1825


# Afrer clear

## integrity ratio by region

In [48]:
%time
procentag_region = spark.sql("""
    SELECT region, COUNT(region) as count, COUNT(region) / {} as procent, COUNT( DISTINCT artist) as artist_count
    FROM charts
    GROUP BY region
    ORDER BY count DESC;""".format(full_data*200))
procentag_region = procentag_region.toPandas()

with open(results_path + "procentag_region.pkl", "wb") as f:
    pickle.dump(procentag_region, f)

procentag_region_filtr = pd.concat([procentag_region.head(5), procentag_region[procentag_region["region"] == "Poland"] , procentag_region.tail(5)])
df_styled = procentag_region_filtr.style.background_gradient()
dfi.export(df_styled, images_path + "procentag_region.png")
df_styled

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 11 µs


,region,count,procent,artist_count
0,Brazil,364516,0.998674,1764
1,Argentina,364326,0.998153,1754
2,United States,364157,0.997690,2068
3,Austria,363810,0.996740,3035
4,Australia,362181,0.992277,1558
26,Poland,358189,0.981340,2528
64,Russia,100767,0.276074,1041
65,Ukraine,100766,0.276071,1010
66,Bulgaria,94834,0.259819,1033
67,South Korea,59576,0.163222,531


In [18]:
# df.sample(False, 0.005).toPandas().to_csv(results_path + "sample_3.csv", index=False)

## artist_counts

In [47]:
artist_counts = df.groupBy('artist').agg(count('title').alias('song_count')).orderBy(col('song_count').desc())
artist_counts_pd = artist_counts.toPandas()
with open(results_path + "artist_counts.pkl", "wb") as f:
    pickle.dump(artist_counts_pd, f)

In [65]:
regions = procentag_region_filtr[:3]["region"].to_list()+["Poland"]
region_top_artist = spark.sql("""
    SELECT artist, region, COUNT(artist) as count
    FROM charts
    WHERE region IN {}
    GROUP BY region, artist
    ORDER BY count DESC;
""".format(tuple(regions)))
region_top_artist_df = region_top_artist.toPandas()
with open(results_path + "region_top_artist.pkl", "wb") as f:
    pickle.dump(region_top_artist_df, f)
region_top_artist_df

,artist,region,count
0,Marília Mendonça,Brazil,11059
1,Juice WRLD,United States,10524
2,Post Malone,United States,10436
3,Zé Neto & Cristiano,Brazil,9385
4,Henrique & Juliano,Brazil,8914
...,...,...,...
8109,La Roux,Poland,1
8110,Soft Cell,United States,1
8111,"Manu Gavassi, Voyou",Brazil,1
8112,Ellie Nelson,United States,1


## Add year, mont, top 10

In [85]:
df_songs = spark.sql("""
    SELECT title, rank, date, artist, url, region, chart, trend, streams, year(date) as year, month(date) as month, rank<10 as top10
    FROM charts;""")
df = df_songs
df.show()
df.createOrReplaceTempView("charts")

+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+----+-----+-----+
|               title|rank|      date|              artist|                 url|   region| chart|        trend|streams|year|month|top10|
+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+----+-----+-----+
|Chantaje (feat. M...|   1|2017-01-01|             Shakira|https://open.spot...|Argentina|top200|SAME_POSITION| 253019|2017|    1| true|
|Vente Pa' Ca (fea...|   2|2017-01-01|        Ricky Martin|https://open.spot...|Argentina|top200|      MOVE_UP| 223988|2017|    1| true|
|Reggaetón Lento (...|   3|2017-01-01|                CNCO|https://open.spot...|Argentina|top200|    MOVE_DOWN| 210943|2017|    1| true|
|              Safari|   4|2017-01-01|J Balvin, Pharrel...|https://open.spot...|Argentina|top200|SAME_POSITION| 173865|2017|    1| true|
|         Shaky Shaky|   5|2017-01-01|   

## total streams

In [88]:
strem_in_years = spark.sql('''
SELECT year, month, sum(streams) as streams
FROM charts
GROUP BY year, month
ORDER BY year, month;
''')
strem_per_yers_pd = strem_in_years.toPandas()
with open(results_path + "strem_per_year.pkl", "wb") as f:
    pickle.dump(strem_per_yers_pd, f)

## most_popoular_aritst_in_year

In [89]:
most_popoular_aritst_in_year = spark.sql("""
SELECT year, artist, sum(streams) as streams
FROM charts
GROUP BY year, artist
ORDER BY year, streams DESC;
""")
with open(results_path + "most_popoular_aritst_in_year.pkl", "wb") as f:
    pickle.dump(most_popoular_aritst_in_year.toPandas(), f)


## market_share_development 

In [93]:
market_share_development = spark.sql("""
SELECT year, region, sum(streams) as streams
FROM charts
WHERE region != 'Global'
GROUP BY year, region
ORDER BY year, region;
""")
market_share_development_pd = market_share_development.toPandas()
with open(results_path + "market_share_development.pkl", "wb") as f:
    pickle.dump(market_share_development_pd, f)

## avg rank by regon

In [5]:
avg_rank_by_region_pd = spark.sql("""
SELECT region, avg(rank) as avg_rank
FROM charts
GROUP BY region
ORDER BY avg_rank;
""")
avg_rank_by_region_pd = avg_rank_by_region_pd.toPandas()
with open(results_path + "avg_rank_by_region_of_best_song.pkl", "wb") as f:
    pickle.dump(avg_rank_by_region_pd, f)

## best songs

In [ ]:
best_song = spark.sql("""
SELECT title, sum(streams) as streams
FROM charts
GROUP BY title
ORDER BY streams DESC;
""")
best_song_pd = best_song.toPandas()
with open(results_path + "best_song.pkl", "wb") as f:
    pickle.dump(best_song_pd, f)

ERROR:root:KeyboardInterrupt while sending command.][Stage 8:>   (0 + 0) / 26]6]
Traceback (most recent call last):
  File "/home/olek/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/olek/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [16]:
best_song = spark.sql("""
SELECT title, count(title) as count, sum(streams) as streams, avg(rank) as avg_rank
FROM charts
GROUP BY title
ORDER BY streams DESC;
""")
best_song_pd = best_song.toPandas()
with open(results_path + "best_song.pkl", "wb") as f:
    pickle.dump(best_song_pd, f)

In [39]:
best_songs_hisotry = spark.sql("""
SELECT title, date, rank, streams
FROM charts
WHERE title IN {};
""".format(tuple(best_song_pd["title"].tolist()[:5])))
best_songs_hisotry_pd = best_songs_hisotry.toPandas()
with open(results_path + "best_songs_hisotry.pkl", "wb") as f:
    pickle.dump(best_songs_hisotry_pd, f)

# avg rank by region of best song

In [9]:
best_song_titles = best_song_pd["title"].to_list()[:100]

In [10]:
avg_rank_by_region_pd = spark.sql("""
SELECT region, avg(rank) as avg_rank
FROM charts
WHERE title IN {}
GROUP BY region
ORDER BY avg_rank;
""".format(tuple(best_song_titles)))
avg_rank_by_region_pd = avg_rank_by_region_pd.toPandas()
with open(results_path + "avg_rank_by_region_of_best_song.pkl", "wb") as f:
    pickle.dump(avg_rank_by_region_pd, f)


# Artist count

In [18]:
# artist_count = spark.sql("""
# # SELECT artist,
# #     count(artist) as count,
# #     sum(streams) as streams,
# #     avg(rank) as avg_rank,
# #     avg(rank) as avg_rank,
# #     max(rank) as max_rank,
# #     min(rank) as min_rank,
# #     sum(rank)/count(artist) as avg_rank_per_song,
# #     sum(streams)/count(artist) as avg_streams_per_song, 
# # FROM charts
# # GROUP BY artist
# # ORDER BY count DESC;
# # """)
artist_count = spark.sql("""
SELECT artist,
    count(artist) as count
FROM charts
GROUP BY artist
ORDER BY count DESC;
""")

artist_count_pd = artist_count.toPandas()
with open(results_path + "artist_count.pkl", "wb") as f:
    pickle.dump(artist_count_pd, f)

In [22]:
df.show()

+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+
|               title|rank|      date|              artist|                 url|   region| chart|        trend|streams|
+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+
|Chantaje (feat. M...|   1|2017-01-01|             Shakira|https://open.spot...|Argentina|top200|SAME_POSITION| 253019|
|Vente Pa' Ca (fea...|   2|2017-01-01|        Ricky Martin|https://open.spot...|Argentina|top200|      MOVE_UP| 223988|
|Reggaetón Lento (...|   3|2017-01-01|                CNCO|https://open.spot...|Argentina|top200|    MOVE_DOWN| 210943|
|              Safari|   4|2017-01-01|J Balvin, Pharrel...|https://open.spot...|Argentina|top200|SAME_POSITION| 173865|
|         Shaky Shaky|   5|2017-01-01|        Daddy Yankee|https://open.spot...|Argentina|top200|      MOVE_UP| 153956|
|         Traicionera|   6|2017-01-01|  

In [5]:
top_one_songs = spark.sql("""
SELECT DISTINCT title, artist
FROM charts
WHERE rank = 1;
""")         
top_one_songs_pd = top_one_songs.toPandas()
top_one_songs_pd


,title,artist
0,Hukutaan,Arttu Lindeman
1,THE SCOTTS,"THE SCOTTS, Travis Scott, Kid Cudi"
2,Colocao,Nicki Nicole
3,Energie,"Ronnie Flex, Frenna"
4,Tamagotchi,"TACONAFIDE, Quebonafide, Taco Hemingway"
...,...,...
2501,Off The Grid,Kanye West
2502,Longue vie,"Sofiane, Ninho, Hornet La Frappe"
2503,Geekd,Daniil
2504,Nössö,"Petri Nygård, Vesku Jokinen"


In [7]:
best_of_top_one_songs = spark.sql("""
SELECT title,
    count(title) as count,
    sum(streams) as streams,
    avg(streams) as avg_streams,  
    avg(rank) as avg_rank
FROM charts
WHERE title in {}
GROUP BY title
ORDER BY streams DESC;
""".format(tuple(top_one_songs_pd["title"].to_list())))

top_rank_one_songs_pd = best_of_top_one_songs.toPandas()
with open(results_path + "top_rank_one_songs.pkl", "wb") as f:
    pickle.dump(top_rank_one_songs_pd, f)


ERROR:root:KeyboardInterrupt while sending command.                (0 + 0) / 26]
Traceback (most recent call last):
  File "/home/olek/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/olek/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

## rank ML models

In [46]:
df.show()

+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+
|               title|rank|      date|              artist|                 url|   region| chart|        trend|streams|
+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+
|Chantaje (feat. M...|   1|2017-01-01|             Shakira|https://open.spot...|Argentina|top200|SAME_POSITION| 253019|
|Vente Pa' Ca (fea...|   2|2017-01-01|        Ricky Martin|https://open.spot...|Argentina|top200|      MOVE_UP| 223988|
|Reggaetón Lento (...|   3|2017-01-01|                CNCO|https://open.spot...|Argentina|top200|    MOVE_DOWN| 210943|
|              Safari|   4|2017-01-01|J Balvin, Pharrel...|https://open.spot...|Argentina|top200|SAME_POSITION| 173865|
|         Shaky Shaky|   5|2017-01-01|        Daddy Yankee|https://open.spot...|Argentina|top200|      MOVE_UP| 153956|
|         Traicionera|   6|2017-01-01|  

In [7]:
with open(results_path + "best_song.pkl", "rb") as f:
    best_song_pd = pickle.load(f)


In [8]:
sonts_to_model = spark.sql("""
SELECT title, artist, rank, streams, date, region
FROM charts
WHERE title in {}
""".format(tuple(best_song_pd[:10]["title"].to_list())))
sonts_to_model_pd = sonts_to_model.toPandas()
sonts_to_model_pd

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler
from pyspark.ml.evaluation import RegressionEvaluator 
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg, col, format_number
# from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, SparkSession, Row

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import requests
import time

In [ ]:
indexers = [StringIndexer(inputCol=c, outputCol='idx_' + c) for c in ['title', 'artist', 'region']]
encoders = [OneHotEncoder(inputCol='idx_' + c, outputCol='ohe_' + c) for c in ['title', 'artist', 'region']]

assembler_before_scaling = VectorAssembler(
    inputCols=['ohe_title', 'ohe_artist', 'ohe_region'],
    outputCol='assembled_features'
)

streams_assembler = VectorAssembler(inputCols=['streams'], outputCol='vec_streams')
rank_assembler = VectorAssembler(inputCols=['rank'], outputCol='vec_rank')

streams_scaler = MinMaxScaler(inputCol='vec_streams', outputCol='scaled_streams')
rank_scaler = MinMaxScaler(inputCol='vec_rank', outputCol='scaled_rank')

assembler_final = VectorAssembler(
    inputCols=['assembled_features', 'scaled_streams'],
    outputCol='features'
)

all_stages = indexers + encoders + [assembler_before_scaling, streams_assembler, rank_assembler, streams_scaler, rank_scaler, assembler_final]

pipeline = Pipeline(stages=all_stages)
df_new = pipeline.fit(df).transform(df)

In [ ]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import DoubleType

# def extract_first_element(v):
#     return float(v[0])

extract_first_element_udf = udf(lambda x: float(x[0]), DoubleType())
select_df = df_new.withColumn('label', extract_first_element_udf(col('scaled_rank')))
select_df = select_df.select(['features', 'label'])
train_data, test_data = select_df.randomSplit([0.8, 0.2], seed=1234)


In [ ]:
lr = LinearRegression(featuresCol='features', labelCol='label')
lr_model = lr.fit(train_data)
predictions = lr_model.transform(test_data)
evaluator = RegressionEvaluator(labelCol='label', predictionCol='prediction', metricName='rmse')
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")
predictions.select('features', 'label', 'prediction').show()


24/06/25 01:18:02 WARN DAGScheduler: Broadcasting large task binary with size 25.4 MiB
ERROR:root:KeyboardInterrupt while sending command.                (0 + 4) / 26]
Traceback (most recent call last):
  File "/home/olek/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/olek/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 